This code is adapted from Tasha Snow's ERA5 downloading script: https://github.com/tsnow03/Landsat_SST_algorithm/blob/main/ERADownload.ipynb

In [ ]:
%pip install ecmwf-api-client
%pip install cdsapi

In [ ]:
import calendar
from ecmwfapi import ECMWFDataServer # Need a ecmwf user name and password first
import cdsapi

The CDS API can be a bit picky with inputs from ERA5, so first-time users are encouraged to use the online request form (https://cds.climate.copernicus.eu/datasets/reanalysis-era5-single-levels?tab=download) to automatically generate a code for their API request, to ensure that the syntax is correct.

In [ ]:
c = cdsapi.Client()

def retrieve_era5():
    """      
       A function to demonstrate how to iterate efficiently over several years and months etc    
       for a particular interim_request.     
       Change the variables below to adapt the iteration to your needs.
       You can use the variable 'target' to organise the requested data in files as you wish.
       In the example below the data are organised in files per month. (eg "interim_daily_201510.grb")
    """
    yearStart = 2020
    yearEnd = 2020
    monthStart = 3
    monthEnd = 4
    dayStart = 1
    dayEnd = 30
    for year in list(range(yearStart, yearEnd + 1)):
        for month in list(range(monthStart, monthEnd + 1)):
            for day in list(range(dayStart, dayEnd + 1)):
                startDy = '%02d' % (day)
                startMo = '%02d' % (month)
                startYr = '%04d' % (year)
                target = "/home/jovyan/tmp/era5_SWE_%04d%02d%02d.nc" % (year, month, day)
                era5_request(startYr, startMo, startDy, target)

def era5_request(startYr, startMo, startDy, target):
    """      
        An ERA-5 request for analysis pressure level data.
        Change the keywords below to adapt it to your needs.
        (eg to add or to remove  levels, parameters, times etc)
        Request cost per day is 112 fields, 14.2326 Mbytes
    """
    c.retrieve(
    'reanalysis-era5-single-levels',
    {
        'product_type':['reanalysis'],
        'data_format':'netcdf',
        'variable':['snow_depth', 'snow_density'],
        'year':[startYr],
        'month':[startMo],
        'day':[startDy],
        'time':['12:00']
    },
    target)
        
if __name__ == '__main__':
    retrieve_era5()

In [ ]:
import netCDF4 as netcdf
import xarray as xr
import rioxarray as rxr

from os import listdir
from os.path import isfile, join

In [ ]:
tmp_path = "/home/jovyan/tmp/"
era5_files = [join(tmp_path,f) for f in listdir(tmp_path) if isfile(join(tmp_path, f))]

In [ ]:
len(era5_files)

In [ ]:
ds = xr.open_mfdataset(era5_files)
ds['valid_time']

In [ ]:
era5_monthly = ds.resample(valid_time='1ME').mean()

In [ ]:
era5_sd_march = era5_monthly['sd'].isel(valid_time=0).compute()

In [ ]:
era5_sd_march.plot.imshow(vmin=0, vmax=1, cmap='Blues')

In [ ]:
ds

In [ ]:
# Making bounds for Alaska
mask_lon = (ds.longitude >= -168.75+360) & (ds.longitude <= -136.01+360)
mask_lat = (ds.latitude >= 52.64) & (ds.latitude <= 71.59)

# Subset MERRA-2 data to Alaska lats/lons only
era5_alaska = ds.where(mask_lon & mask_lat, drop=True)

In [ ]:
era5_alaska = era5_alaska.compute()

In [ ]:
era5_alaska['sd'].isel(valid_time=0).plot.imshow(vmin=0, vmax=1, cmap="Blues")

In [ ]:
era5_sd_alaska = era5_alaska['sd'].mean(('longitude', 'latitude'))

In [ ]:
era5_sd_alaska.plot()